# Day 22: Reactor Reboot

Operating at these extreme ocean depths has overloaded the submarine's reactor; it needs to be rebooted.

The reactor core is made up of a large 3-dimensional grid made up entirely of cubes, one cube per integer 3-dimensional coordinate (x,y,z). Each cube can be either on or off; at the start of the reboot process, they are all off. (Could it be an old model of a reactor you've seen before?)

To reboot the reactor, you just need to set all of the cubes to either on or off by following a list of reboot steps (your puzzle input). Each step specifies a cuboid (the set of all cubes that have coordinates which fall within ranges for x, y, and z) and whether to turn all of the cubes in that cuboid on or off.

## Actions

In [ ]:
record struct Dim(int min, int max) {
    static public Dim From(int[] nums) => new Dim(nums[0], nums[1]);
    public bool Valid => min <= max;
    public decimal Count => max - min + 1;
    public override string ToString() => $"{min}..{max}";
    public IEnumerable<int> Range => Enumerable.Range(min, max - min + 1);
    public bool Contains(int i) => min <= i && i <= max;
    public bool Inside(Dim d) => d.min <= min && max <= d.max;
    public bool Overlaps(Dim d) => d.min <= max && d.max >= min;
    public (Dim,Dim,Dim) Section(Dim d) {
        var l = new Dim(min, d.min-1);
        var g = new Dim(d.max+1, max);
        if (l.Valid) {
            if (g.Valid) return (l, d, g);
            return (l, new Dim(d.min, max), g);
        }
        if (g.Valid) return (l, new Dim(min, d.max), g);
        return (l, this, g);        
    }
}

record struct Cube(Dim x, Dim y, Dim z) {
    static public Cube From(int[] nums)
        => new Cube(Dim.From(nums[0..2]), new Dim(nums[2], nums[3]), new Dim(nums[4], nums[5]));
    public decimal Count
        => x.Count * y.Count * z.Count;
    public bool Overlaps(Cube c)
        => x.Overlaps(c.x) && y.Overlaps(c.y) && z.Overlaps(c.z);
    public IEnumerable<(int x, int y, int z)> Range { get {
            var yRange = y.Range;
            var zRange = z.Range;
            return from xPos in x.Range
                from yPos in yRange
                from zPos in zRange
                select (xPos, yPos, zPos);
        } }
    public IEnumerable<Cube> Minus(Cube d) {
        if (!Overlaps(d)) return new[] { this };

        var result = new List<Cube>();
        var (xl, xm, xg) = x.Section(d.x);
        var (yl, ym, yg) = y.Section(d.y);
        var (zl, zm, zg) = z.Section(d.z);
        
        if (x.Inside(d.x)) {
            if (y.Inside(d.y)) {
                if (zl.Valid) result.Add(new Cube(x, y, zl));
                if (zg.Valid) result.Add(new Cube(x, y, zg));
                return result;
            }
            if (z.Inside(d.z)) {
                if (yl.Valid) result.Add(new Cube(x, yl, z));
                if (yg.Valid) result.Add(new Cube(x, yg, z));
                return result;
            }
            if (zl.Valid) result.Add(new Cube(x, y, zl));
            if (yl.Valid) result.Add(new Cube(x, yl, zm));
            if (yg.Valid) result.Add(new Cube(x, yg, zm));
            if (zg.Valid) result.Add(new Cube(x, y, zg));
            return result;
        }
        if (y.Inside(d.y)) {
            if (z.Inside(d.z)) {
                if (xl.Valid) result.Add(new Cube(xl, y, z));
                if (xg.Valid) result.Add(new Cube(xg, y, z));
                return result;
            }
            if (zl.Valid) result.Add(new Cube(x, y, zl));
            if (xl.Valid) result.Add(new Cube(xl, y, zm));
            if (xg.Valid) result.Add(new Cube(xg, y, zm));
            if (zg.Valid) result.Add(new Cube(x, y, zg));
            return result;
        }            
        if (zl.Valid) result.Add(new Cube(x, y, zl));
        if (yl.Valid) result.Add(new Cube(x, yl, zm));
        if (xl.Valid) result.Add(new Cube(xl, ym, zm));
        if (xg.Valid) result.Add(new Cube(xg, ym, zm));
        if (yg.Valid) result.Add(new Cube(x, yg, zm));
        if (zg.Valid) result.Add(new Cube(x, y, zg));
        return result;
    }
    public override string ToString()
        => $"{x}/{y}/{z}";
}

record struct Action(bool state, Cube cube) {
    public override string ToString()
        => $"{state} {cube}";
}

static var seps = new [] {' ', '=', '.', ',', 'x', 'y', 'z'};

static Action Parse(string line) {
    var parts = line.Split(seps, StringSplitOptions.RemoveEmptyEntries);
    var nums = parts[1..].Select(int.Parse).ToArray();

    return new Action(parts[0] == "on", Cube.From(nums));
}

Parse("on x=-20..26,y=-36..17,z=-47..7").ToString()

True -20..26/-36..17/-47..7

## Input

In [ ]:
using System.IO;

/* var input = new [] {
        "on x=10..12,y=10..12,z=10..12",
        "on x=11..13,y=11..13,z=11..13",
        "off x=9..11,y=9..11,z=9..11",
        "on x=10..10,y=10..10,z=10..10"
    }; // */
// var input = File.ReadAllLines(@"day-22.sample1");
// var input = File.ReadAllLines(@"day-22.sample2");
var input = File.ReadAllLines(@"day-22.input");

var actions = input.Select(Parse).ToArray();

// display(actions.Select(a => actions.Where(b => a!=b && a.cube.Overlaps(b.cube)).Select(b => $"{a}|{b}")));

actions.Length

420

## Part 1

Execute the reboot steps. Afterward, considering only cubes in the region x=-50..50,y=-50..50,z=-50..50, how many cubes are on?

In [ ]:
var initDim = new Dim(-50,50);
var cubes = initDim.Range.Select(x => initDim.Range.Select(y => initDim.Range.Select(z => false).ToArray()).ToArray()).ToArray();
var initRegion = new Cube(initDim,initDim,initDim);

foreach (var a in actions) {
    var c = a.cube;
    if (!c.Overlaps(initRegion)) continue;

    foreach (var (x, y, z) in c.Range)
        if (initDim.Contains(x) && initDim.Contains(y) && initDim.Contains(z))
        cubes[x+50][y+50][z+50] = a.state;
}

cubes.Sum(x => x.Sum(y => y.Count(z => z)))

650099

## Part 2

Now that the initialization procedure is complete, you can reboot the reactor.

Starting with all cubes off, run all of the reboot steps for all cubes in the reactor.

In [ ]:
var cubes = new List<Cube>();

foreach (var a in actions) {
    var overlaps = cubes.Where(a.cube.Overlaps).ToArray();
    if (a.state) {
        if (overlaps.Any()) {
            var adds = overlaps.Aggregate(new[] { a.cube }, 
                (a, c) => a.SelectMany(s => s.Minus(c)).ToArray());
            cubes.AddRange(adds);
        } else
            cubes.Add(a.cube);
    } else {
        foreach (var o in overlaps)
            cubes.Remove(o);
        var adds = overlaps.Aggregate(Array.Empty<Cube>(), 
            (l, c) => l.Concat(c.Minus(a.cube)).ToArray());
        cubes.AddRange(adds);
    }
}

// cubes.Count()
cubes.Sum(c => c.Count)

1254011191104293